Устанавливаем HMMER

In [ ]:
%%bash
wget http://eddylab.org/software/hmmer/hmmer-3.3.2.tar.gz
tar xf hmmer-3.3.2.tar.gz
cd hmmer-3.3.2

In [ ]:
%%bash
cd hmmer-3.3.2
./configure
make
make install

Скачиваем Pfam
По ftp доступны разны версии базы. Нас интересует последняя в формате hmm

In [3]:
!wget http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam35.0/Pfam-A.hmm.gz

--2024-06-15 19:13:53--  http://ftp.ebi.ac.uk/pub/databases/Pfam/releases/Pfam35.0/Pfam-A.hmm.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 293000230 (279M) [application/x-gzip]
Saving to: ‘Pfam-A.hmm.gz’

Pfam-A.hmm.gz       100%[===================>] 279.43M   714KB/s    in 6m 49s  

2024-06-15 19:20:42 (700 KB/s) - ‘Pfam-A.hmm.gz’ saved [293000230/293000230]



Извлекаем из архива

In [4]:
!gunzip Pfam-A.hmm.gz

Создаем нужные для hmmer'a файлы

In [5]:
!hmmpress Pfam-A.hmm

Working...    done.
Pressed and indexed 19632 HMMs (19632 names and 19632 accessions).
Models pressed into binary file:   Pfam-A.hmm.h3m
SSI index for binary model file:   Pfam-A.hmm.h3i
Profiles (MSV part) pressed into:  Pfam-A.hmm.h3f
Profiles (remainder) pressed into: Pfam-A.hmm.h3p


In [6]:
!head -n 40 Pfam-A.hmm

HMMER3/f [3.1b2 | February 2015]
NAME  1-cysPrx_C
ACC   PF10417.12
DESC  C-terminal domain of 1-Cys peroxiredoxin
LENG  40
ALPH  amino
RF    no
MM    no
CONS  yes
CS    yes
MAP   yes
DATE  Thu Nov  4 19:20:02 2021
NSEQ  40
EFFN  17.426758
CKSUM 4086680297
GA    21.1 21.1;
TC    21.1 21.1;
NC    21 21;
BM    hmmbuild HMM.ann SEED.ann
SM    hmmsearch -Z 61295632 -E 1000 --cpu 4 HMM pfamseq
STATS LOCAL MSV       -7.5463  0.71948
STATS LOCAL VITERBI   -7.8624  0.71948
STATS LOCAL FORWARD   -4.3303  0.71948
HMM          A        C        D        E        F        G        H        I        K        L        M        N        P        Q        R        S        T        V        W        Y   
            m->m     m->i     m->d     i->m     i->i     d->m     d->d
  COMPO   2.28046  4.31208  2.83393  2.63913  3.90855  2.69988  3.89812  3.33401  2.56310  2.85023  3.99954  3.22924  2.52123  2.90328  3.31238  2.94055  2.70512  2.59551  3.49266  3.82715
          2.68618  4.42225  2.77519  2.7312

Смотрим статистику

In [7]:
!hmmstat Pfam-A.hmm.h3m | head -n 20

# hmmstat :: display summary statistics for a profile file
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#
# idx  name                 accession        nseq eff_nseq      M relent   info p relE compKL
# ---- -------------------- ------------ -------- -------- ------ ------ ------ ------ ------
1      1-cysPrx_C           PF10417.12         40    17.43     40   1.37   1.32   1.29   0.09
2      120_Rick_ant         PF12574.11          2     0.45    238   0.59   0.60   0.52   0.02
3      12TM_1               PF09847.12          7     2.28    449   0.59   0.62   0.51   0.09
4      14-3-3               PF00244.23        149     2.50    222   0.59   0.59   0.52   0.05
5      17kDa_Anti_2         PF16998.8           6     0.99    116   0.59   0.57   0.52   0.03
6      2-Hacid_dh           PF00389.33         78    

The columns are:

**idx**
The index of this profile, numbering each on in the file starting from 1.

**name**
The name of the profile.

**accession**
The optional accession of the profile, or "-" if there is none.

**nseq**
The number of sequences that the profile was estimated from.

**eff_nseq**
The effective number of sequences that the profile was estimated from, after HMMER applied an effective sequence number calculation such as the default entropy weighting.

**M**
The length of the model in consensus residues (match states).

**relent**
Mean relative entropy per match state, in bits. This is the expected (mean) score per consensus position. This is what the default entropy-weighting method for effective sequence number estimation focuses on, so for default HMMER3 models, you expect this value to reflect the default target for entropy-weighting.

**info**
Mean information content per match state, in bits. Probably not useful. Information content is a slightly different calculation than relative entropy.

**p relE**
Mean positional relative entropy, in bits. This is a fancier version of the per-match-state relative entropy, taking into account the transition (insertion/deletion) probabilities; it may be a more accurate estimation of the average score contributed per model consensus position.

**compKL**
Kullback-Leibler distance between the model's overall average residue composition and the default background frequency distribution. The higher this number, the more biased the residue composition of the profile is. Highly biased profiles can slow the HMMER3 acceleration pipeline, by causing too many nonhomologous sequences to pass the filters.

Индексируем файл

In [8]:
!hmmfetch --index Pfam-A.hmm.h3m

Working...    done.
Indexed 19632 HMMs (19632 names and 19632 accessions).
SSI index written to file Pfam-A.hmm.h3m.ssi


In [9]:
input_file = 'input.txt'
output_file = 'Pfam_domains.txt'

# Открываем файл для чтения
with open(input_file, 'r') as file:
    # Считываем строки из файла
    lines = file.readlines()

# Извлекаем все слова, начинающиеся с "PF" из каждой строки
pf_words = []
for line in lines:
    words = line.replace(';', ' ').replace(',', ' ').split()  # Удаляем пунктуацию и разбиваем строку на слова
    for word in words:
        if word.startswith('PF'):
            pf_words.append(word)

# Записываем найденные слова в столбик в файл Pfam_domains.txt
with open(output_file, 'w') as file:
    for word in pf_words:
        file.write(word + '\n')

print("Слова, начинающиеся с 'PF', успешно записаны в файл Pfam_domains.txt.")


Слова, начинающиеся с 'PF', успешно записаны в файл Pfam_domains.txt.


In [10]:
input_file = 'Pfam_domains.txt'
output_file = 'Unique_Pfam_domains.txt'

# Считываем уникальные слова начинающиеся с "PF" из файла
unique_pf_words = set()
with open(input_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        word = line.strip()
        unique_pf_words.add(word)

# Записываем уникальные слова в файл Unique_Pfam_domains.txt
with open(output_file, 'w') as file:
    for word in unique_pf_words:
        file.write(word + '\n')

print("Уникальные слова, начинающиеся с 'PF', успешно записаны в файл Unique_Pfam_domains.txt.")

Уникальные слова, начинающиеся с 'PF', успешно записаны в файл Unique_Pfam_domains.txt.


Скачиваем протеом


In [ ]:
!head protein.faa

>XP_034419599.1 fam-b protein [Plasmodium berghei ANKA]
MRISILKFVFFSIIICSFEYVKNELYFVNDRGIYLERDVINFKNNKILADVDNQFDFNEFYESTLSLASQLGDCIEGNKE
IAYLQSIIDSHIKKHKENNTLPDLNNVDKKTKKLIDQLRKELEEIKKELNNKRSSEVEIQPIKDKIITKKNKNNSVSEQE
DFKQLENEGNFLEINDDNFKDEYNEITSSSNYKKLKINRKLKKENKKLVKKFMVFMVSHFVILGSGGGYLILLVIPCMIS
VIKNCWKSLKLSFKKSEI
>XP_034419600.1 conserved rodent malaria protein, unknown function [Plasmodium berghei ANKA]
MVPSSMINNDIPLAKNACVIRPKVKANLHISLKNRKNAKDGAANISHNLQNHVENRIKERSLYYAAKEKYYRNKGKKYDK
LRQSVECGESRILKEYYHSRNRIKRCSRKHKKHNYEKCSVCKSLVQRYLVLGENANKIVEGINERTNDIMCNSICDTDPN
SKMNELIKEETEEKYSTIKRITPTAPVRKFDIAENIKNNDITKLPKNESLDGLYINNEKDLDSYDLSNMKNIVLSHDKKL
GDTYIISDKNIICDLENKVEQKGNNYPLVKCENQFNENQSCANIESNDISKKKHMYTIFTDLNVSEIIDLMNRKPSNITN


In [ ]:
!hmmstat Pfam-A.hmm

In [12]:
!hmmsearch Z-alpha.hmm protein.faa


Error: File existence/permissions problem in trying to open HMM file Z-alpha.hmm.
HMM file Z-alpha.hmm not found (nor an .h3m binary of it)



## Поиск


In [ ]:
!hmmsearch --tblout search.csv Pfam-A.hmm protein.faa

## Теперь надо фильтровать

In [14]:
def read_domains(file_name):
    """ Читает файл с уникальными доменами Pfam и возвращает их в виде множества. """
    with open(file_name, 'r') as file:
        domains = {line.strip() for line in file}
    return domains

def filter_csv_by_domains(csv_file, domains, output_file):
    """ Проверяет строки csv файла на наличие любого из доменов и записывает их в новый файл. """
    import csv

    with open(csv_file, mode='r', newline='', encoding='utf-8') as inp_file, \
         open(output_file, mode='w', newline='', encoding='utf-8') as out_file:
        reader = csv.reader(inp_file)
        writer = csv.writer(out_file)

        for row in reader:
            row_text = ','.join(row)  # Преобразуем строку списка в одну строчку текста
            if any(domain in row_text for domain in domains):
                writer.writerow(row)

def main():
    domains_file = 'Unique_Pfam_domains.txt'
    source_csv_file = 'search.csv'
    output_csv_file = 'filtered_results.csv'

    # Загрузка уникальных доменов Pfam из файла
    domains = read_domains(domains_file)

    # Фильтрация CSV файла
    filter_csv_by_domains(source_csv_file, domains, output_csv_file)
    print("Фильтрация завершена, результаты записаны в", output_csv_file)
main()

Фильтрация завершена, результаты записаны в filtered_results.csv


In [15]:
import csv
def process_filtered_results(input_csv, output_file):

    with open(input_csv, mode='r', newline='', encoding='utf-8') as inp_file, \
         open(output_file, mode='w', newline='', encoding='utf-8') as out_file:
        reader = csv.reader(inp_file)
        writer = csv.writer(out_file)

        writer.writerow(['Id','Gene', 'Family'])  # Записываем заголовки столбцов

        for row in reader:
            row_text = ' '.join(row)  # Соединяем строку, если она разделена запятой
            elements = row_text.split()  # Делим строку по пробелам
            if len(elements) >= 4:  # Проверяем, есть ли минимум 4 элемента
                id = elements[0] # Первый элемент - id
                gene = elements[2]  # Третий элемент
                family = elements[3]  # Четвертый элемент
                writer.writerow([id, gene, family])  # Записываем в файл

def main():
    filtered_csv_file = 'filtered_results.csv'
    output_csv_file = 'gene_family_results.csv'

    # Обработка полученных результатов и запись в новый файл
    process_filtered_results(filtered_csv_file, output_csv_file)
    print("Обработка завершена, результаты записаны в", output_csv_file)

main()


Обработка завершена, результаты записаны в gene_family_results.csv


In [16]:
def remove_duplicates(input_csv, output_csv):
    import csv
    unique_genes = {}  # Словарь для хранения уникальных генов и их семейств

    with open(input_csv, mode='r', newline='', encoding='utf-8') as inp_file:
        reader = csv.reader(inp_file)
        header = next(reader, None)  # Пропускаем заголовок

        for row in reader:
            gene = row[0]
            family = row[1]
            if gene not in unique_genes:  # Если ген не в повторяющихся, добавляем его
                unique_genes[gene] = family

    with open(output_csv, mode='w', newline='', encoding='utf-8') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(header)  # Записываем заголовки столбцов
        for gene, family in unique_genes.items():
            writer.writerow([gene, family])  # Записываем уникальные пары ген-семейство

def main():
    input_csv_file = 'gene_family_results.csv'
    output_csv_file = 'unique_gene_family_results.csv'

    # Удаление дубликатов по полю ген и запись в новый файл
    remove_duplicates(input_csv_file, output_csv_file)
    print("Дубликаты удалены, результаты записаны в", output_csv_file)

main()


Дубликаты удалены, результаты записаны в unique_gene_family_results.csv
